In [11]:
import requests
import json
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [214]:
class Assets():
    def __init__(self, isin):
        self.isin = isin

    def _xetra_history(self, start_date, end_date, periode='day'):
        df = pd.DataFrame(pd.bdate_range(start_date, end_date, freq='B'), columns=['date'])
        df.set_index('date', inplace=True)

        limit = (datetime.strptime(end_date, '%Y-%m-%d').year - \
                 datetime.strptime(start_date, '%Y-%m-%d').year+1)*253

        api_url = 'https://api.boerse-frankfurt.de/data/price_history?limit='+str(limit)+ \
                '&offset=0&isin=' + str(self.isin) + '&mic=XETR&minDate='+ \
                str(start_date)+'&maxDate='+str(end_date)
        
        req = requests.get(api_url)
        json_data = req.json()['data']
            
        df_prices = pd.DataFrame(json_data)
        df_prices.set_index('date', inplace=True)
        df_prices.sort_index(inplace=True)
            
        df = pd.merge(df, df_prices, how='left', left_index=True, right_index=True)
        df.dropna(how='all', inplace=True)          
        return df
    
    
    def _langschwarz_history(self, start_date, end_date, periode):
        pass
        
        
    def hist_prices(self, start_date, end_date, periode='day', data_source='xetra'):
        if data_source == 'xetra':
            return self._xetra_history(start_date, end_date)
        
        if data_source == 'langschwarz':
            return self._langschwarz_history(start_date, end_date)
       
    
    def get_ticker(self, start_time='08:00', end_time='22:00'):
        today_date = datetime.today().strftime('%Y-%m-%d')
        start_h, start_min = start_time.split(':')[0], start_time.split(':')[-1]
        end_h, end_min = end_time.split(':')[0], end_time.split(':')[-1]
        
        api_url = 'https://api.boerse-frankfurt.de/data/tick_data?limit=2500&offset=0&isin='+ \
                str(self.isin)+'&mic=XETR&minDateTime='+today_date+'T'+str(start_h)+'%3A'+ \
                str(start_min)+'%3A00.000Z&maxDateTime='+today_date+'T'+str(end_h)+'%3A'+ \
                str(end_min)+'%3A00.000Z'
        
        req = requests.get(api_url)
        json_data = req.json()['ticks']
        df_ticker = pd.DataFrame(json_data)
        df_ticker.set_index('time', inplace=True)
        return df_ticker
    
    
    def get_price(self):
        api_url = 'https://api.boerse-frankfurt.de/data/price_information?isin=' + \
        str(self.isin) + '&mic=XETR'
        
        req = requests.get(api_url, stream=True)
        for line in req.iter_lines():
            if line:
                price = float(json.loads(line[5:].decode('utf-8')).get('lastPrice'))
                return price             
       
    
    def price_stream(self):
        api_url = 'https://api.boerse-frankfurt.de/data/price_information?isin=' + \
        str(self.isin) + '&mic=XETR'
        
        req = requests.get(api_url, stream=True)
        for line in req.iter_lines():     
            if line:
                price = float(json.loads(line[5:].decode('utf-8')).get('lastPrice'))
                yield price
        
    
    def get_bid_ask(self):
        api_url = 'https://api.boerse-frankfurt.de/data/bid_ask_overview?isin=' + \
        str(self.isin) + '&mic=XETR'
        
        req = requests.get(api_url, stream=True)
        for line in req.iter_lines(decode_unicode=True):
            if line:
                json_data = json.loads(line[5:]).get('data')[0]
                
                bid_price = float(json_data.get('bidPrice'))
                ask_price = float(json_data.get('askPrice'))
                return bid_price, ask_price
            
            
    def bid_ask_stream(self):
        api_url = 'https://api.boerse-frankfurt.de/data/bid_ask_overview?isin=' + \
        str(self.isin) + '&mic=XETR'
        
        req = requests.get(api_url, stream=True)
        for line in req.iter_lines(decode_unicode=True):
            if line:
                json_data = json.loads(line[5:]).get('data')[0]
                
                bid_price = float(json_data.get('bidPrice'))
                ask_price = float(json_data.get('askPrice'))
                yield [bid_price, ask_price ]       
              

    def get_fee(self):
        api_url = 'https://api.boerse-frankfurt.de/data/fees_etp?isin=' + str(self.isin)
        req = requests.get(api_url, stream=True)        
        total_expense_perc = req.json().get('totalExpensePercent')
        return total_expense_perc                   
       

## Historical prices for a single asset

In [204]:
asset_prices = Assets('DE000A0H08D2').hist_prices(start_date='2020-01-01', end_date='2020-03-01')
asset_prices.head()

,open,close,high,low,turnoverPieces,turnoverEuro
date,,,,,,
2020-01-02,19.442,19.484,19.568,19.442,36747.0,717800.38
2020-01-03,19.366,19.378,19.384,19.312,15210.0,294449.00
2020-01-06,19.228,19.234,19.246,19.112,79552.0,1525667.13
2020-01-07,19.378,19.408,19.430,19.354,24226.0,469576.97
2020-01-08,19.268,19.436,19.436,19.258,20883.0,402873.16


## Historical closing prices of multiple assets

In [205]:
start_date, end_date = '2020-01-01', '2020-03-01'
df = pd.DataFrame(pd.bdate_range(start_date, end_date, freq='B'), columns=['date'])
df.set_index('date', inplace=True)

isin_list = ['IE00B42Z5J44', 'DE000A0H08D2']

for isin in isin_list:
    df[isin] = Assets(isin).hist_prices(start_date, end_date)['close']
    
df.head()

,IE00B42Z5J44,DE000A0H08D2
date,,
2020-01-01,NaN,NaN
2020-01-02,50.264,19.484
2020-01-03,49.888,19.378
2020-01-06,49.988,19.234
2020-01-07,50.346,19.408


## Most recent price of an asset

In [194]:
price = Assets('IE00B42Z5J44').get_price()
print(price)

40.858


## Price data live stream of an asset

In [206]:
for price in Assets('DE000A0H08D2').price_stream():
    print(price)
    break

15.718


## Most recent bid and ask price

In [210]:
bid_price, ask_price = Assets('IE00B42Z5J44').get_bid_ask()
print(bid_price, ask_price)

40.952 41.02


## Spread data live stream of an asset

In [217]:
for spread in Assets('IE00B42Z5J44').bid_ask_stream():
    bid_price, ask_price = spread[0], spread[1]
    print(bid_price, ask_price)
    break

41.01 41.064


## Today's ticker data

In [123]:
ticker = Assets('IE00B42Z5J44').get_ticker(start_time='09:34', end_time='17:00')
ticker.head()

,price,turnover,turnoverInEuro
time,,,
2020-03-24T12:01:51+01:00,41.111,79.0,3247.769
2020-03-24T12:01:50+01:00,41.107,1565.0,64332.455
2020-03-24T12:01:48+01:00,41.106,97.0,3987.282
2020-03-24T12:01:22+01:00,41.049,2992.0,122818.608
2020-03-24T11:57:53+01:00,41.000,5000.0,205000.000


## Total expense percentage of a fund or ETF

In [207]:
fee = Assets('IE00B42Z5J44').get_fee()
print(fee)

0.64
